In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import swat
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

# DWH Data Best Yükleme

In [2]:
col_list = ['tarih',
            'kanal',
            'grup_adi',
            'ana_kategori_adi',
            'kategori_adi',
            'marka_adi',
            'urun_adi',
            'en_guncel_kod',
            'gerceklesen_satis_adedi',
            'tahmin_adet',
            'tahmin_ust_sinir',
            'tahmin_alt_sinir',
            'koli_ici_adet',
            'tahmin_koli',
            'mape',
            'gerceklesen_koli',
            'indirim_yuzdesi',
            'aktivite_tipi',
            'tahmin_edilme_yontemi',
            'datanin_etiketlendigi_algorit',
            'tahmin_sapmasi',
            'kg_unit',
            'tl_unit',
            'kg',
            'tl',
            'tahmin_koli_alt_sinir',
            'tahmin_koli_ust_sinir',
            'aktivite_etkisi',
            'from']

In [3]:
sigma1_5_raw = pd.read_excel("../results/36_12_siparis/Tahmin Sonuçları (Tüm Kombinasyonlar)_final_sigma1_5.xlsx")
sigma1_75_raw = pd.read_excel("../results/36_12_siparis/Tahmin Sonuçları (Tüm Kombinasyonlar)_final_sigma1_75.xlsx")
sigma2_0_raw = pd.read_excel("../results/36_12_siparis/Tahmin Sonuçları (Tüm Kombinasyonlar)_final_sigma2_0.xlsx")

In [4]:
sigma1_5 = sigma1_5_raw.copy()
sigma1_75 = sigma1_75_raw.copy()
sigma2_0 = sigma2_0_raw.copy()

In [5]:
sigma1_5 = sigma1_5[~(sigma1_5["urun_adi"].str.contains("TÜM"))].reset_index(drop=True)
sigma1_75 = sigma1_75[~(sigma1_75["urun_adi"].str.contains("TÜM"))].reset_index(drop=True)
sigma2_0 = sigma2_0[~(sigma2_0["urun_adi"].str.contains("TÜM"))].reset_index(drop=True)

In [6]:
sigma_map = pd.read_excel("../reports/_Demand Sensing 4 Aylık Tahmin 10072021_mostdx.xlsx", sheet_name="sigma_map")
sigma_map.rename(columns={"Kanal": "kanal", "En Güncel Kod": "en_guncel_kod"}, inplace=True)

In [7]:
sigma1_5 = sigma1_5.merge(sigma_map, how="left")

In [8]:
sigma1_75 = sigma1_75.merge(sigma_map, how="left")

In [9]:
sigma2_0 = sigma2_0.merge(sigma_map, how="left")

In [10]:
sigma1_5 = sigma1_5[sigma1_5["type"] == "sigma1,5"]
sigma1_75 = sigma1_75[sigma1_75["type"] == "sigma1,75"]
sigma2_0 = sigma2_0[sigma2_0["type"] == "sigma2,0"]

In [11]:
final_data_with_june_sadece_predictionlar3 = pd.concat([sigma1_5, sigma1_75, sigma2_0], ignore_index=True)
final_data_with_june_sadece_predictionlar3.drop(["type", "tl_unit", "kg_unit", "tl", "kg"], axis=1, inplace=True)

In [12]:
final_data_with_june_sadece_predictionlar3["from"] = "row"

In [13]:
fiyat_df = pd.read_csv("../data/fiyat_list.csv")
fiyat_df = fiyat_df.drop_duplicates(subset=["en_guncel_kod"], keep="last")
fiyat_df = fiyat_df[["en_guncel_kod", "fiyat"]]
fiyat_df.rename(columns={"fiyat": "tl_unit"}, inplace=True)

In [14]:
koli_agirlik_df = pd.read_excel("../data/koli_birim_agirliklari.xls")
koli_agirlik_df = koli_agirlik_df.drop_duplicates(subset=["Malzeme"], ignore_index=True)
koli_agirlik_df = koli_agirlik_df[["Malzeme", "Net Ağırlık"]]
koli_agirlik_df.rename(columns={"Malzeme": "en_guncel_kod", "Net Ağırlık": "kg_unit"}, inplace=True)

In [15]:
final_data_with_june_sadece_predictionlar3 = final_data_with_june_sadece_predictionlar3.merge(fiyat_df, how="left")
final_data_with_june_sadece_predictionlar3 = final_data_with_june_sadece_predictionlar3.merge(koli_agirlik_df, how="left")

In [16]:
final_data_with_june_sadece_predictionlar3["kg"] = final_data_with_june_sadece_predictionlar3["tahmin_koli"] * final_data_with_june_sadece_predictionlar3["kg_unit"]
final_data_with_june_sadece_predictionlar3["tl"] = final_data_with_june_sadece_predictionlar3["tahmin_koli"] * final_data_with_june_sadece_predictionlar3["tl_unit"]
final_data_with_june_sadece_predictionlar3 = final_data_with_june_sadece_predictionlar3[col_list]

In [17]:
kanal_all = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
kanal_all["grup_adi"] = "TÜM GRUPLAR"
kanal_all["ana_kategori_adi"] = "TÜM ANA KATEGORİLER"
kanal_all["kategori_adi"] = "TÜM KATEGORİLER"
kanal_all["marka_adi"] = "TÜM MARKALAR"
kanal_all["en_guncel_kod"] = "TÜM SKULAR"
kanal_all["urun_adi"] = "TÜM ÜRÜNLER"
kanal_all["tahmin_edilme_yontemi"] = "N/A"
kanal_all["datanin_etiketlendigi_algorit"] = "N/A"
kanal_all["aktivite_etkisi"] = "Var"

In [18]:
kanal_all["from"] = "kanal_all"

In [19]:
grup_all = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
grup_all["ana_kategori_adi"] = "TÜM ANA KATEGORİLER"
grup_all["kategori_adi"] = "TÜM KATEGORİLER"
grup_all["marka_adi"] = "TÜM MARKALAR"
grup_all["en_guncel_kod"] = "TÜM SKULAR"
grup_all["urun_adi"] = "TÜM ÜRÜNLER"
grup_all["tahmin_edilme_yontemi"] = "N/A"
grup_all["datanin_etiketlendigi_algorit"] = "N/A"
grup_all["aktivite_etkisi"] = "Var"

In [20]:
grup_all["from"] = "grup_all"

In [21]:
kontrol = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).agg({"kategori_adi": "nunique"}).reset_index()
idx_delete = kontrol[kontrol["kategori_adi"] == 1].index

In [22]:
ana_kategori_all = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
ana_kategori_all["kategori_adi"] = "TÜM KATEGORİLER"
ana_kategori_all["marka_adi"] = "TÜM MARKALAR"
ana_kategori_all["en_guncel_kod"] = "TÜM SKULAR"
ana_kategori_all["urun_adi"] = "TÜM ÜRÜNLER"
ana_kategori_all["tahmin_edilme_yontemi"] = "N/A"
ana_kategori_all["datanin_etiketlendigi_algorit"] = "N/A"
ana_kategori_all["aktivite_etkisi"] = "Var"

In [23]:
ana_kategori_all = ana_kategori_all[~(ana_kategori_all.index.isin(idx_delete))]
ana_kategori_all["from"] = "ana_kategori_all"

In [24]:
kontrol = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", 
                                                              "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).agg({"marka_adi": "nunique"}).reset_index()
idx_delete = kontrol[kontrol["marka_adi"] == 1].index

In [25]:
kategori_all = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", 
                                                                   "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()

kategori_all["marka_adi"] = "TÜM MARKALAR"
kategori_all["en_guncel_kod"] = "TÜM SKULAR"
kategori_all["urun_adi"] = "TÜM ÜRÜNLER"
kategori_all["tahmin_edilme_yontemi"] = "N/A"
kategori_all["datanin_etiketlendigi_algorit"] = "N/A"
kategori_all["aktivite_etkisi"] = "Var"

In [26]:
kategori_all = kategori_all[~(kategori_all.index.isin(idx_delete))]
kategori_all["from"] = "kategori_all"

In [27]:
kontrol = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", 
                                                              "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).agg({"en_guncel_kod": "nunique"}).reset_index()
idx_delete = kontrol[kontrol["en_guncel_kod"] == 1].index

In [28]:
marka_all = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", 
                                                                "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()

marka_all["en_guncel_kod"] = "TÜM SKULAR"
marka_all["urun_adi"] = "TÜM ÜRÜNLER"
marka_all["tahmin_edilme_yontemi"] = "N/A"
marka_all["datanin_etiketlendigi_algorit"] = "N/A"
marka_all["aktivite_etkisi"] = "Var"

In [29]:
marka_all = marka_all[~(marka_all.index.isin(idx_delete))]
marka_all["from"] = "marka_all"

# Hiyerarşik detay kırılımları ile

In [30]:
grup_detail = []
for idx in final_data_with_june_sadece_predictionlar3.grup_adi.unique():
    tmp = final_data_with_june_sadece_predictionlar3[final_data_with_june_sadece_predictionlar3["grup_adi"] == idx]
    tmp = tmp.groupby(["tarih", "kanal", "grup_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
    tmp["ana_kategori_adi"] = "TÜM ANA KATEGORİLER"
    tmp["kategori_adi"] = "TÜM KATEGORİLER"
    tmp["marka_adi"] = "TÜM MARKALAR"
    tmp["en_guncel_kod"] = "TÜM SKULAR"
    tmp["urun_adi"] = "TÜM ÜRÜNLER"
    tmp["tahmin_edilme_yontemi"] = "N/A"
    tmp["datanin_etiketlendigi_algorit"] = "N/A"
    tmp["aktivite_etkisi"] = "Var"
    grup_detail.append(tmp)
grup_detail = pd.concat(grup_detail, ignore_index=True)

In [31]:
grup_detail["from"] = "grup_detail"

In [32]:
ana_kategori_detail = []
for idx in final_data_with_june_sadece_predictionlar3.ana_kategori_adi.unique():
    tmp = final_data_with_june_sadece_predictionlar3[final_data_with_june_sadece_predictionlar3["ana_kategori_adi"] == idx]
    kontrol = tmp.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).agg({"kategori_adi": "nunique"}).reset_index().kategori_adi.unique()[0]
    tmp = tmp.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
    if kontrol == 1:
        pass
    else:
        tmp["kategori_adi"] = "TÜM KATEGORİLER"
        tmp["marka_adi"] = "TÜM MARKALAR"
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
        tmp["aktivite_etkisi"] = "Var"
        ana_kategori_detail.append(tmp)
ana_kategori_detail = pd.concat(ana_kategori_detail, ignore_index=True)

In [33]:
ana_kategori_detail["from"] = "ana_kategori_detail"

In [34]:
liste = final_data_with_june_sadece_predictionlar3.drop_duplicates(subset=["kanal", "grup_adi", "ana_kategori_adi", "kategori_adi"])[["kanal", "grup_adi", "ana_kategori_adi", "kategori_adi"]].reset_index(drop=True)
kategori_detail = []
for idx in liste.index:
    tmp = final_data_with_june_sadece_predictionlar3[(final_data_with_june_sadece_predictionlar3["kanal"] == liste["kanal"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["grup_adi"] == liste["grup_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["ana_kategori_adi"] == liste["ana_kategori_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["kategori_adi"] == liste["kategori_adi"][idx])]
    kontrol = len(tmp["marka_adi"].unique())
    if kontrol == 1:
        pass
    else:
        tmp = tmp.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
        tmp["marka_adi"] = "TÜM MARKALAR"
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
        tmp["aktivite_etkisi"] = "Var"
        kategori_detail.append(tmp)
kategori_detail = pd.concat(kategori_detail, ignore_index=True)

In [35]:
kategori_detail["from"] = "kategori_detail"

In [36]:
liste = final_data_with_june_sadece_predictionlar3.drop_duplicates(subset=["kanal", "grup_adi", "ana_kategori_adi", 
                                                                           "kategori_adi", "marka_adi"])[["kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi"]].reset_index(drop=True)
marka_detail = []
for idx in liste.index:
    tmp = final_data_with_june_sadece_predictionlar3[(final_data_with_june_sadece_predictionlar3["kanal"] == liste["kanal"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["grup_adi"] == liste["grup_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["ana_kategori_adi"] == liste["ana_kategori_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["kategori_adi"] == liste["kategori_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["marka_adi"] == liste["marka_adi"][idx])]
    kontrol = len(tmp["en_guncel_kod"].unique())
    if kontrol == 1:
        pass
    else:
        tmp = tmp.groupby(["tarih", "kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
        tmp["aktivite_etkisi"] = "Var"
        marka_detail.append(tmp)
marka_detail = pd.concat(marka_detail, ignore_index=True)

In [37]:
marka_detail["from"] = "marka_detail"

In [38]:
tum_gruplar_ana_kategori = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
tum_gruplar_ana_kategori["grup_adi"] = "TÜM GRUPLAR"
tum_gruplar_ana_kategori["kategori_adi"] = "TÜM KATEGORİLER"
tum_gruplar_ana_kategori["marka_adi"] = "TÜM MARKALAR"
tum_gruplar_ana_kategori["en_guncel_kod"] = "TÜM SKULAR"
tum_gruplar_ana_kategori["urun_adi"] = "TÜM ÜRÜNLER"
tum_gruplar_ana_kategori["tahmin_edilme_yontemi"] = "N/A"
tum_gruplar_ana_kategori["datanin_etiketlendigi_algorit"] = "N/A"
tum_gruplar_ana_kategori["aktivite_etkisi"] = "Var"

In [39]:
tum_gruplar_ana_kategori["from"] = "tum_gruplar_ana_kategori"

In [40]:
tum_gruplar_kategori = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "ana_kategori_adi", "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
tum_gruplar_kategori["grup_adi"] = "TÜM GRUPLAR"
tum_gruplar_kategori["marka_adi"] = "TÜM MARKALAR"
tum_gruplar_kategori["en_guncel_kod"] = "TÜM SKULAR"
tum_gruplar_kategori["urun_adi"] = "TÜM ÜRÜNLER"
tum_gruplar_kategori["tahmin_edilme_yontemi"] = "N/A"
tum_gruplar_kategori["datanin_etiketlendigi_algorit"] = "N/A"
tum_gruplar_kategori["aktivite_etkisi"] = "Var"

In [41]:
tum_gruplar_kategori["from"] = "tum_gruplar_kategori"

In [42]:
liste = final_data_with_june_sadece_predictionlar3.drop_duplicates(subset=["kanal", "ana_kategori_adi", 
                                                                           "kategori_adi", "marka_adi"])[["kanal", "ana_kategori_adi", 
                                                                                                          "kategori_adi", "marka_adi"]].reset_index(drop=True)

In [43]:
tum_gruplar_marka = []
for idx in liste.index:
    tmp = final_data_with_june_sadece_predictionlar3[(final_data_with_june_sadece_predictionlar3["kanal"] == liste["kanal"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["ana_kategori_adi"] == liste["ana_kategori_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["kategori_adi"] == liste["kategori_adi"][idx]) & 
                                                     (final_data_with_june_sadece_predictionlar3["marka_adi"] == liste["marka_adi"][idx])]
    kontrol = len(tmp["en_guncel_kod"].unique())
    if kontrol == 1:
        pass
    else:
        tmp = tmp.groupby(["tarih", "kanal", "ana_kategori_adi", "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
        tmp["grup_adi"] = "TÜM GRUPLAR"
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
        tmp["aktivite_etkisi"] = "Var"
        tum_gruplar_marka.append(tmp)
tum_gruplar_marka = pd.concat(tum_gruplar_marka, ignore_index=True)

In [44]:
tum_gruplar_marka["from"] = "tum_gruplar_marka"

In [45]:
tum_gruplar_en_guncel_kod = final_data_with_june_sadece_predictionlar3.groupby(["tarih", "kanal", "ana_kategori_adi", "kategori_adi", "marka_adi", "en_guncel_kod", "urun_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
tum_gruplar_en_guncel_kod["grup_adi"] = "TÜM GRUPLAR"
tum_gruplar_en_guncel_kod["tahmin_edilme_yontemi"] = "N/A"
tum_gruplar_en_guncel_kod["datanin_etiketlendigi_algorit"] = "N/A"
tum_gruplar_en_guncel_kod["aktivite_etkisi"] = "Var"

In [46]:
tum_gruplar_en_guncel_kod["from"] = "tum_gruplar_en_guncel_kod"

In [47]:
kanal_all = kanal_all[final_data_with_june_sadece_predictionlar3.columns.to_list()]
grup_all = grup_all[final_data_with_june_sadece_predictionlar3.columns.to_list()]
ana_kategori_all = ana_kategori_all[final_data_with_june_sadece_predictionlar3.columns.to_list()]
kategori_all = kategori_all[final_data_with_june_sadece_predictionlar3.columns.to_list()]
marka_all = marka_all[final_data_with_june_sadece_predictionlar3.columns.to_list()]
grup_detail = grup_detail[final_data_with_june_sadece_predictionlar3.columns.to_list()]
ana_kategori_detail = ana_kategori_detail[final_data_with_june_sadece_predictionlar3.columns.to_list()]
kategori_detail = kategori_detail[final_data_with_june_sadece_predictionlar3.columns.to_list()]
marka_detail = marka_detail[final_data_with_june_sadece_predictionlar3.columns.to_list()]
tum_gruplar_ana_kategori = tum_gruplar_ana_kategori[final_data_with_june_sadece_predictionlar3.columns.to_list()]
tum_gruplar_kategori = tum_gruplar_kategori[final_data_with_june_sadece_predictionlar3.columns.to_list()]
tum_gruplar_marka = tum_gruplar_marka[final_data_with_june_sadece_predictionlar3.columns.to_list()]
tum_gruplar_en_guncel_kod = tum_gruplar_en_guncel_kod[final_data_with_june_sadece_predictionlar3.columns.to_list()]

In [48]:
final_data_with_june_sadece_predictionlar4 = pd.concat([final_data_with_june_sadece_predictionlar3,
                                                        kanal_all,
                                                        grup_all,
                                                        ana_kategori_all,
                                                        kategori_all,
                                                        marka_all,
                                                        grup_detail,
                                                        ana_kategori_detail,
                                                        kategori_detail,
                                                        marka_detail,
                                                        tum_gruplar_ana_kategori,
                                                        tum_gruplar_kategori,
                                                        tum_gruplar_marka,
                                                        tum_gruplar_en_guncel_kod], axis=0, ignore_index=True)

In [49]:
final_data_with_june_sadece_predictionlar4.drop_duplicates(subset=final_data_with_june_sadece_predictionlar4.columns.to_list(), 
                                                           ignore_index=True, 
                                                           inplace=True)

In [51]:
historic_data = pd.read_csv("../data/_historic_data_for_graph.csv")

In [52]:
final_data_with_june_sadece_predictionlar4.columns = [i[:-2] if len(i) > 30 else i for i in final_data_with_june_sadece_predictionlar4.columns]
final_data_with_june_sadece_predictionlar4["en_guncel_kod"] = final_data_with_june_sadece_predictionlar4["en_guncel_kod"].apply(lambda x: "TÜM SKULAR" if x == "TÜM SKU'LAR" else x)

In [53]:
final_data_with_june_sadece_predictionlar4.drop("aktivite_etkisi", axis=1, inplace=True)
grup_and_sku_list = final_data_with_june_sadece_predictionlar4.drop_duplicates(subset=["grup_adi", "en_guncel_kod"], ignore_index=True)[["grup_adi", "en_guncel_kod"]]
final_data_with_june_sadece_predictionlar4["aktivite_etkisi"] = np.nan
for idx in grup_and_sku_list.index:
    akt = historic_data[(historic_data["Grup Adı"] == grup_and_sku_list["grup_adi"][idx]) & 
                        (historic_data["En Güncel Kod"] == grup_and_sku_list["en_guncel_kod"][idx])]["Aktivite Tipi"].unique()
    ind = historic_data[(historic_data["Grup Adı"] == grup_and_sku_list["grup_adi"][idx]) & 
                        (historic_data["En Güncel Kod"] == grup_and_sku_list["en_guncel_kod"][idx])]["İndirim Yüzdesi"].unique()
    indexes = final_data_with_june_sadece_predictionlar4[(final_data_with_june_sadece_predictionlar4["grup_adi"] == grup_and_sku_list["grup_adi"][idx]) & 
                                                         (final_data_with_june_sadece_predictionlar4["en_guncel_kod"] == grup_and_sku_list["en_guncel_kod"][idx])].index
    if (len(akt) == 1 and len(ind) == 1) and (akt[0] == 0 and ind[0] == -1):
        final_data_with_june_sadece_predictionlar4.loc[list(indexes), "aktivite_etkisi"] = "Yok"
    else:
        final_data_with_june_sadece_predictionlar4.loc[list(indexes), "aktivite_etkisi"] = "Var"
final_data_with_june_sadece_predictionlar4.aktivite_etkisi.fillna("Var", inplace=True)

In [54]:
final_data_with_june_sadece_predictionlar4["aktivite_tipi"] = final_data_with_june_sadece_predictionlar4["aktivite_tipi"].apply(lambda x: "In-out" if x=="In&Out" else x)

In [55]:
try:
    final_data_with_june_sadece_predictionlar4.drop(columns="from", axis=1, inplace=True)
except:
    pass

In [56]:
final_data_with_june_sadece_predictionlar4.drop_duplicates(ignore_index=True, inplace=True)

In [57]:
access_ = True
while access_:
    try:
        conn = swat.CAS('yhtrcl-sasccnt1.yildiz.domain', 5570, username='tunahan.aktas', password='34m153294T...')
        access_ = False
        print("Accessed!")
    except:
        print("Got error. Trying to reconnect...")
        continue

Accessed!


In [58]:
ds_table = conn.CASTable("DEMAND_SENSING_RESULTS", caslib='DSENS_P', replace=True)
ds_table.table.dropTable(quiet=True)

+ Elapsed: 0.0378s, user: 0.0104s, sys: 0.00689s, mem: 0.829mb

In [59]:
conn.upload(data=final_data_with_june_sadece_predictionlar4, 
            casout={'caslib':'DSENS_P', 'name':"DEMAND_SENSING_RESULTS", 'promote':True})

NOTE: Cloud Analytic Services made the uploaded file available as table DEMAND_SENSING_RESULTS in caslib DSENS_P.
NOTE: The table DEMAND_SENSING_RESULTS has been created in caslib DSENS_P from binary data uploaded to Cloud Analytic Services.


[caslib]

 'DSENS_P'

[tableName]

 'DEMAND_SENSING_RESULTS'

[casTable]

 CASTable('DEMAND_SENSING_RESULTS', caslib='DSENS_P')

+ Elapsed: 23.2s, user: 2.98s, sys: 1.91s, mem: 163mb